# Progetto di Social Computing

a.a. 2022-2023

## Attività preliminari

### Librerie e costanti

In [2]:
# Caricamento delle librerie
import os, json, random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# Cartelle di salvataggio
data_folder = "./data"
out_folder = "./out"

### Funzioni

In [3]:
# Salvataggio in locale
def serialize_json(folder, filename, data):
    if not os.path.exists(folder):
        os.makedirs(folder, exist_ok=True)
    
    with open(f"{folder}/{filename}", "w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent = 4)
        f.close()
    print(f"Data serialized to path: {folder}/{filename}")

In [4]:
# Lettura da locale
def read_json(path):
    if os.path.exists(path):
        with open(path, "r", encoding="utf-8") as file:
            data = json.load(file)
        print(f"Data read from path: {path}")
        return data
    else:
        print(f"No data found at path: {path}")
        return {}

## Caricamento del dataset

Si carica il dataset fornito, frammento della combinazione dei dataset [FEVER](https://fever.ai/dataset/fever.html) e [e-FEVER](https://doi.org/10.3929/ethz-b-000453826).

In [32]:
# Si carica e si mostra il dataset fornito
df = pd.read_csv("./group_9.csv")
display(df)

,id,statement,explanation_human,explanation_model,label
0,51526,Hush (2016 film) was produced by Jason Blum.,Hush (2016 film) was produced by Trevor Macy a...,The evidence says that the film was produced b...,REFUTES
1,77465,Winter's Tale was released in 1987.,Winter's Tale was released in 2014.,The claim is that Winter's Tale was released i...,REFUTES
2,166632,Anne Rice was born in the United States of Ame...,"Anne Rice was born in New Orleans, Louisiana, ...",The claim is that Anne Rice was born in the Un...,SUPPORTS


## Creazione degli HITs

Con il dataset fornito, vogliamo creare creare dodici HITs aventi le seguenti caratteristiche:

1. contiene 3 elementi;
2. ogni elemento è dotato di 4 attributi:
   1. `id`: identificatore dello statement;
   2. `statement`: testo dello statement;
   3. `explanation`: testo della spiegazione;
   4. `label`: etichetta della spiegazione.
3. per due elementi su tre vale che `explanation` = `explanation_model`;
4. per un elemento su tre vale che `explanation` = `explanation_human`;
5. la posizione dei tre elementi deve essere casuale.

In [14]:
# Funzione per l'interpretazione delle righe del dataset
def parse_row(row, isGold = False):
    parsed = {
        "id" : row.id,
        "statement" : row.statement,
        "explanation" : row.explanation_model,
        "label" : row.label,
        "isGold" : isGold
    }

    # Se è una "domanda d'oro", la spiegazione deve essere quella fornita da un essere umano
    if isGold:
        parsed["explanation"] = row.explanation_human
    
    return parsed

In [31]:
# Creiamo tutte le possibili permutazioni dei 3 elementi secondo quanto stabilito dalla consegna
all_HITs = []

# Iteriamo sulle spiegazioni fornite dai modelli di machine learning
for model_exp in df.itertuples():
    model_HITs = []

    # Iteriamo sugli altri "statements"
    for other_stat in df.itertuples():
        if other_stat.id != model_exp.id:
            # Inseriamo la versione con la "domanda d'oro" per la spiegazione del modello (model_exp)
            other_model_HIT = [parse_row(model_exp), parse_row(model_exp, True), parse_row(other_stat)]
            # Inseriamo la versione con la "domanda d'oro" per questo statement (other_stat)
            other_gold_HIT = [parse_row(model_exp), parse_row(other_stat, True), parse_row(other_stat)]

            # Riordiniamo pseudo-casualmente gli elementi
            random.shuffle(other_model_HIT)
            random.shuffle(other_gold_HIT)
            # Aggiungiamo le HIT create a quelle relative a questa spiegazione (model_exp)
            model_HITs.append(other_model_HIT)
            model_HITs.append(other_gold_HIT)
    
    # Si concatena il tutto a tutti gli HITs possibili
    all_HITs += model_HITs

# Riposizioniamo gli elementi della lista in maniera pseudo-causale
random.shuffle(all_HITs)

# Salviamo la lista degli HIT creata
serialize_json(data_folder, "all_HITs.json", all_HITs)

Data serialized to path: ./data/all_HITs.json


### Adattamento degli HITs per Crowd_Frame

Ristrutturiamo gli HITs generati precedentemente in modo da renderli leggibili al software [Crowd_Frame](https://github.com/Miccighel/Crowd_Frame).

In [5]:
def generate_random_string(n_chars):
    random_string = ""
    for i in range(n_chars):
        # Generiamo un carattere minuscolo
        random_integer = random.randint(97, 97 + 26 - 1)
        flip_bit = random.randint(0, 1)
        # Lo rendiamo casualmente maiuscolo
        random_integer = random_integer - 32 if flip_bit == 1 else random_integer
        # Concateniamo alla stringa casuale
        random_string += chr(random_integer)
    return random_string

In [9]:
# Carichiamo gli HITs generati
raw_HITs = read_json(data_folder+"/all_HITs.json")

HITs = []
id = 0
documents_number = 3

for raw_HIT in raw_HITs:
    # Creiamo l'HIT definitiva
    HIT = {
        "unit_id" : "unit_"+str(id),
        "token_input" : generate_random_string(10),
        "token_output" : generate_random_string(10),
        "documents_number" : documents_number
    }
    # Aggiungiamo i documenti
    documents = []
    for element in raw_HIT:
        # Discriminiamo le domande d'oro
        pre = "G_" if element["isGold"] else "N_"
        # Creiamo il documento con i suoi attributi e lo aggiungiamo
        document = {
            "id" : pre+str(element["id"]),
            "statement" : element["statement"],
            "label" : element["label"],
            "explanation" : element["explanation"]
        }
        documents.append(document)
    # Associamo i documenti riformattati all'HIT
    HIT["documents"] = documents
    # Aggiungiamo l'HIT all'insieme delle HIT
    HITs.append(HIT)
    # Incremetiamo il valore dell'id
    id += 1

# Si esporta il tutto oome file JSON
serialize_json(out_folder, "hits.json", HITs)

Data read from path: ./data/all_HITs.json
Data serialized to path: ./out/hits.json


In [21]:
workers_time_id_resp = pd.read_csv("./result/secondo_progetto_social_computing/Dataframe/workers_answers.csv", 
                         usecols=['worker_id','time_submit_parsed','doc_id','doc_truthfulness-1_value','doc_truthfulness-2_value'])
#display(workers_time_id_resp)

workers_n = workers_time_id_resp.to_numpy()
workers_n_parsed = pd.DataFrame(workers_n, columns=["worker_id", "time", "doc_id", "doc_1_val","doc_2_val"])
# otteniamo le risposte per i primi worker e per i secondi workers 
workers_first_task = workers_n_parsed[["worker_id","doc_id","doc_1_val"]]
workers_second_task = workers_n_parsed[["worker_id","doc_id","doc_2_val"]]
# abbiamo notato che ci sono stati degli aggiornamenti della query, i valori 
# non ho trovato una veloce soluzione quindi lo droppo manualmente 
# problema nella riga 16 in conflitto con la riga 19 -> prevale la riga 19 per via del timestamp 
# problema nella riga 17 in conflitto con la riga 20 -> prevale la riga 20 (stessa motv)
# problema nella riga 18-21-22-23 -> prevale la riga 23 
# probelma nella riga 24 in conflitto con la riga 27-28 -> prevale la riga 24
# problema nella riga 25 in conflitto con 29 -> prevale la riga 25
# problema nella riga 26 in conflitto con 30 -> prevale la riga 26
workers_first_task = workers_first_task.drop([16,17,18,21,22,27,28,29,30])
workers_first_task = workers_first_task.reset_index(drop=True)
# print(workers_first_task)

# valutiamo anche le seconde risposte 
# conflitto con riga 3 -> riga 7 --> vince la riga 7
# conflitto con riga 5 -> riga 9 --> vince la riga 5
# conflitto con riga 6 -> riga 8 --> vince la riga 8 
# conflitto con riga 16 -> riga 19 --> vince la riga 19 
# conflitto con riga 17 -> riga 20 --> vince la riga 20 
# conflitto con riga 18 - 21 - 22 - 23 -> vince la riga 23 
# conflitto con riga 24 - 27 - 28 -> vince la riga 24
# conflitto con riga 25 - 29 -> vince la riga 25
# conflitto con riga 26 - 30 -> vince la riga 26 
workers_second_task = workers_second_task.drop([3,9,6,16,17,18,21,22,27,28,29,30]).reset_index(drop=True)
workers_second_task = workers_second_task.set_index('worker_id')
# print(workers_second_task)

#         risposta worker 
# doc_id  
# table document -> N_166632
# ricorda che per il numero di coppie totali possibili -> (n*(n-1))/2
def compute_percent_agreement(doc):
    doc_reset = doc.reset_index(level="worker_id")

    n_rows = doc_reset.shape[0]
    total_pairs = (n_rows*(n_rows-1))/2
    n_equal_pairs = 0
    
    for x in range(n_rows): 
        for y  in range(n_rows):
            if(x < y and doc_reset.at[x, "doc_2_val"] == doc_reset.at[y, "doc_2_val"]): 
                n_equal_pairs += 1
    # percentage agreement 
    #   numero_coppie_in_accordo
    #   ------------------------
    #   numero coppie totali 
    return n_equal_pairs / total_pairs

doc_1 = workers_second_task.loc[workers_second_task['doc_id'] == 'N_166632']             # task -> N_166632
print(compute_percent_agreement(doc_1)) # proof of work
doc_2 = workers_second_task.loc[workers_second_task['doc_id'] == 'G_166632']             # task -> G_166632
doc_3 = workers_second_task.loc[workers_second_task['doc_id'] == 'N_51526']              # task -> N_51526
doc_4 = workers_second_task.loc[workers_second_task['doc_id'] == 'G_51526']              # task -> G_51526
doc_5 = workers_second_task.loc[workers_second_task['doc_id'] == 'N_77465']              # task -> N_77465
doc_6 = workers_second_task.loc[workers_second_task['doc_id'] == 'G_77465']              # task -> G_77465

# numero di volte che è stato annotato -> prendo tutte le istanze dello stesso task dello stesso worker e calcolo quante sono 



# Calcolate la percentuale media di testo annotato per ciascuna spiegazione Ordinate le spiegazioni sulla base di tale parametro
# in questo caso ci serve tempo inizio tempo fine e task 
workers_for_time = pd.read_csv('./result/secondo_progetto_social_computing/Dataframe/workers_dimensions_selection.csv', 
                         usecols=['worker_id','dimension_name','document_id','timestamp_start_parsed','timestamp_end_parsed'])

worker_one = workers_for_time.loc[workers_for_time['worker_id'] == 'A348JKD82WQ6Z']     # worker uno A348JKD82WQ6Z
worker_two = workers_for_time.loc[workers_for_time['worker_id'] == 'A3OAQZM6Q3YJQ1']    # worker due A3OAQZM6Q3YJQ1
worker_tree = workers_for_time.loc[workers_for_time['worker_id'] == 'A1TEEFJDPVEK0L']    # worker tre A1TEEFJDPVEK0L
worker_four = workers_for_time.loc[workers_for_time['worker_id'] == 'A3T2NTPGB3KNDS']   # worker 4 A3T2NTPGB3KNDS
worker_five = workers_for_time.loc[workers_for_time['worker_id'] == 'AYKZJHEV29ZHL']    # worker 5 AYKZJHEV29ZHL
worker_six = workers_for_time.loc[workers_for_time['worker_id'] == 'A2Q51AC4E6I5ZB']    # worker 6 A2Q51AC4E6I5ZB
worker_seven = workers_for_time.loc[workers_for_time['worker_id'] == 'A3W16X5D0VGU0E']  # worker 7 A3W16X5D0VGU0E
worker_eight = workers_for_time.loc[workers_for_time['worker_id'] == 'A348JKD82WQ6Z']   # worker 8 A2Z4OTGC834F3Y
worker_nine = workers_for_time.loc[workers_for_time['worker_id'] == 'A2N1GA8PJDDA6P']   # worker 9 A2N1GA8PJDDA6P
worker_ten = workers_for_time.loc[workers_for_time['worker_id'] == 'AYUF9OHXQK2YT']     # worker 10 AYUF9OHXQK2YT

# ottenuti i dati di inizio e di fine -> fare la differenza


0.6
